## Prepare N Duration Datasets

In [1]:
import os
import pandas as pd
import numpy as np

# Directory where the CSV files are stored
CSV_DIR = 'datasets'

# List of different N values
Ns = [5, 10, 50, 100, 150, 300, 500, 1000, 5000, 10000, 15000, 20000]

tolerance = 0.10  # Define the tolerance

# Load and preprocess the data for common dataframe df1
df1 = pd.read_parquet(os.path.join(CSV_DIR, 'wednesday_cf.parquet'))

# Preprocessing df1 to facilitate filtering based on 'bidirectional_packets' for different values of n
df1 = df1[['flow_key_hash', 'bidirectional_duration_ms', 'label']]

for n in Ns:
    # Load data for each df2 based on n value
    df2 = pd.read_csv(os.path.join(CSV_DIR, f'wednesday_fd_{n}.csv'))

    # Filter df2: remove duplicate 'forward_hash' entries, keeping only the row with the lowest 'id'
    df2.sort_values(by='id', inplace=True)
    df2 = df2.drop_duplicates(subset='flow_key_hash', keep='first')

    # Merge df1 and df2. This adds 'label' and 'bidirectional_packets' from df1 to df2
    merged_df = df2.merge(df1, on='flow_key_hash', how='left', suffixes=('', '_df1'))

    # Filter out rows where bidirectional_duration_ms from df1 is less than n and keep rows with labels
    filtered_df = merged_df[(merged_df['bidirectional_duration_ms_df1'] >= (n * (1 - tolerance))) & merged_df['label'].notna()]

    # Filter based on the new criteria for 'bidirectional_duration_ms'
    # final_df = merged_df[((merged_df['bidirectional_duration_ms_df1'] >= (n * (1 - tolerance))) & 
    #                       (merged_df['bidirectional_duration_ms_df1'] <= (n * (1 + tolerance)))) & 
    #                      merged_df['label'].notna()]

    # Drop rows where a certain label occurs fewer than 100 times
    label_counts = filtered_df['label'].value_counts()
    labels_to_keep = label_counts[label_counts >= 100].index
    final_df = filtered_df[filtered_df['label'].isin(labels_to_keep)]
    
    # Ensure each 'flow_key_hash' is unique, then drop unnecessary columns including 'flow_key_hash'
    final_df = final_df.drop_duplicates(subset='flow_key_hash', keep='first').drop(columns=['bidirectional_duration_ms_df1'])

    # Downcast integers and floats
    for col in final_df.columns:
        col_type = final_df[col].dtype
    
        if np.issubdtype(col_type, np.integer):
            final_df[col] = pd.to_numeric(final_df[col], downcast='integer')
        elif np.issubdtype(col_type, np.floating):
            final_df[col] = pd.to_numeric(final_df[col], downcast='float')    

    # Store the updated dataframe in a CSV file, keeping only those flows that have a label assigned
    final_df.to_parquet(os.path.join(CSV_DIR, f'wednesday_fd_{n}.parquet'), index=False)

In [2]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import json

DAY = "wednesday"

# Initialize an empty list to store data dictionaries
data_list = []

# Initializing the table with updated formatting to match the expanded header.
header = ["DS", "TOTAL", "BENIGN", "ANOMALY", "Anomaly breakdown", "Min Pkts.", "Mean Pkts.", "Max Pkts."]
# rowh = "{:^15} " + "{:^7} " + "{:^7} " + "{:^7} " + "{:^26} "            + "{:^10} " + "{:^10} " + "{:^10} "
# row  = "{:^15} " + "{:^7} " + "{:^7} " + "{:^7} " + "{:<18}  " + "{:>6} " + "{:>10} " + "{:>10} " + "{:>10} "
# sep  = ["-"*15]  + ["-"*7] + ["-"*7] + ["-"*7] + ["-"*26]             + ["-"*10]  + ["-"*10] + ["-"*10]
rowh = "{:^12} " * 4 + "{:^26} "            + "{:^10} " + "{:^12} " + "{:^12} "
row  = "{:^12} " * 4 + "{:<18}  " + "{:>6} " + "{:>10} " + "{:>12} " + "{:>12} "
sep  = ["-"*12]  * 4 + ["-"*26]             + ["-"*10]  + ["-"*12] + ["-"*12]

print(rowh.format(*header))  # This should now work without an IndexError

Ns = ['cf'] + [5, 10, 50, 100, 150, 300, 500, 1000, 5000, 10000, 15000, 20000]

for n in Ns:
    print(rowh.format(*sep))

    # Determine the filename based on the value of n
    filename = f"{DAY}_cf.parquet" if n == 'cf' else f"{DAY}_fd_{n}.parquet"

    # Load the CSV
    csv = pd.read_parquet(os.path.join(CSV_DIR,filename))

    TOTAL = len(csv)
    BENIGN = len(csv[csv["label"] == "BENIGN"])
    ANOMALY = len(csv[(csv["label"] != "BENIGN")])

    # Create and append the summary row for each file
    summary_row = {
        "DS": DAY + '_' + str(n),
        "TOTAL": TOTAL,
        "BENIGN": BENIGN,
        "ANOMALY": ANOMALY,
        "Anomaly breakdown": [],
    }
    
    print(row.format(DAY+'_'+str(n), TOTAL, BENIGN, ANOMALY, "", "", "", "", "")) 
    for label in sorted(csv["label"].unique().tolist()):
        if label in ["BENIGN"]:  # Skip benign and NaN labels
            continue
        # Calculate min, mean, and max packets for each anomaly
        anomaly_data = csv[csv["label"] == label]['bidirectional_packets']
        # min = np.min(anomaly_data)
        # mean = np.mean(anomaly_data)
        # max = np.max(anomaly_data)
        if not anomaly_data.empty:
            min = int(np.min(anomaly_data))  # Convert to int
            mean = float(np.mean(anomaly_data))  # Convert to float
            max = int(np.max(anomaly_data))  # Convert to int
        
        print(row.format("", "", "", "", label, len(csv[csv["label"] == label]), min, "{:.2f}".format(mean), max))
        
        # Append the anomaly details to the "Anomaly breakdown" list
        summary_row["Anomaly breakdown"].append({
            "Anomaly Type": label,
            "Count": len(anomaly_data),
            "Min Packets": min,
            "Mean Packets": mean,
            "Max Packets": max,
        })
    
    data_list.append(summary_row)

# Now, `data_list` contains all the information
# Convert the list to a JSON string and write it to a file
json_data = json.dumps(data_list, indent=4)
with open('results/fd_anomaly_distribution.json', 'w') as file:
    file.write(json_data)

     DS         TOTAL        BENIGN      ANOMALY        Anomaly breakdown      Min Pkts.   Mean Pkts.   Max Pkts.   
------------ ------------ ------------ ------------ -------------------------- ---------- ------------ ------------ 
wednesday_cf    502350       326363       175987                                                                    
                                                    DoS GoldenEye         7917          2        12.14           30 
                                                    DoS Hulk            158680          2         9.20           27 
                                                    DoS Slowhttptest      3707          1         4.72           35 
                                                    DoS Slowloris         5683          2         7.36           27 
------------ ------------ ------------ ------------ -------------------------- ---------- ------------ ------------ 
wednesday_5      7693         729          6964                 

## Sanity check

In [3]:
import pandas as pd

X = 10

# Load the dataset
df = pd.read_parquet('datasets/wednesday_cf.parquet')

# Check how many rows have a certain label and 'bidirectional_duration_ms' higher than or equal to X
benign_flows = df[(df['label'] == 'BENIGN') & (df['bidirectional_duration_ms'] >= X)].shape[0]
goldeneye_flows = df[(df['label'] == 'DoS GoldenEye') & (df['bidirectional_duration_ms'] >= X)].shape[0]
hulk_flows = df[(df['label'] == 'DoS Hulk') & (df['bidirectional_duration_ms'] >= X)].shape[0]
slowhttptest_flows = df[(df['label'] == 'DoS Slowhttptest') & (df['bidirectional_duration_ms'] >= X)].shape[0]
slowloris_flows = df[(df['label'] == 'DoS Slowloris') & (df['bidirectional_duration_ms'] >= X)].shape[0]

# Print the result
print(f"Number of flows with label 'BENIGN' and 'bidirectional_duration_ms' >= {X}: {benign_flows}")
print(f"Number of flows with label 'DoS GoldenEye' and 'bidirectional_duration_ms' >= {X}: {goldeneye_flows}")
print(f"Number of flows with label 'DoS Hulk' and 'bidirectional_duration_ms' >= {X}: {hulk_flows}")
print(f"Number of flows with label 'DoS Slowhttptest' and 'bidirectional_duration_ms' >= {X}: {slowhttptest_flows}")
print(f"Number of flows with label 'DoS Slowloris' and 'bidirectional_duration_ms' >= {X}: {slowloris_flows}")

Number of flows with label 'BENIGN' and 'bidirectional_duration_ms' >= 10: 217922
Number of flows with label 'DoS GoldenEye' and 'bidirectional_duration_ms' >= 10: 7568
Number of flows with label 'DoS Hulk' and 'bidirectional_duration_ms' >= 10: 144510
Number of flows with label 'DoS Slowhttptest' and 'bidirectional_duration_ms' >= 10: 1492
Number of flows with label 'DoS Slowloris' and 'bidirectional_duration_ms' >= 10: 2325


## Convert the values into LaTeX table

In [4]:
import json

# Function to add thousand separators
def add_thousand_sep(number):
    if isinstance(number, float):
        # For floating point numbers, split on decimal point
        integer_part, decimal_part = f"{number:.2f}".split(".")
        integer_part_with_sep = "{:_}".format(int(integer_part)).replace("_", " ")
        return f"{integer_part_with_sep}.{decimal_part}"
    elif isinstance(number, int):
        # For integers, just add the separator
        return "{:_}".format(number).replace("_", " ")
    else:
        # Return the value as it is if it's not a number
        return number

# Load the JSON data from the file
with open('results/fd_anomaly_distribution.json', 'r') as file:
    data_list = json.load(file)

# Start the LaTeX table and define the header
latex_code = """
\\begin{table*}[htbp]
\\scriptsize
\\centering
\\caption{Your Table Caption}
\\renewcommand{\\arraystretch}{0.6} % Reduce spacing
\\begin{tabular}{lrrrrrrrr}
\\toprule
\\textbf{DS} & \\textbf{TOTAL} & \\textbf{BENIGN} & \\textbf{ANOMALY} & \\textbf{Anomaly Type} & \\textbf{Count} & \\textbf{Min Pckts.} & \\textbf{Mean Pckts.} & \\textbf{Max Pckts.} \\\\
\\midrule
"""

# Helper to determine how many anomaly breakdowns are present for a given DS
def count_anomalies(entry):
    return len(entry.get("Anomaly breakdown", []))

# Iterate over each entry in the data list to populate the table rows
for entry in data_list:
    # Extract DS value and transform it according to the specified rules
    ds_value = entry['DS']
    if ds_value == "wednesday_cf":
        ds_label = "CF"
    elif ds_value.startswith("wednesday_"):
        ds_number = ds_value.split("_")[-1]  # Extract the number part
        ds_label = f"FD={ds_number}ms"
    else:
        ds_label = ds_value  # Fallback to the original DS value if none of the above rules apply

    anomaly_count = count_anomalies(entry)

    # Format the numbers with thousand separator
    total_formatted = add_thousand_sep(entry['TOTAL'])
    benign_formatted = add_thousand_sep(entry['BENIGN'])
    anomaly_formatted = add_thousand_sep(entry['ANOMALY'])

    # Add the summary row for each dataset
    # Check if there's more than one type of anomaly for multirow
    anomaly_breakdown = entry['Anomaly breakdown']
    if len(anomaly_breakdown) > 1:
        latex_code += f"\\multirow{{{len(anomaly_breakdown)}}}{{*}}{{{ds_label}}} & "
        latex_code += f"\\multirow{{{len(anomaly_breakdown)}}}{{*}}{{{add_thousand_sep(entry['TOTAL'])}}} & "
        latex_code += f"\\multirow{{{len(anomaly_breakdown)}}}{{*}}{{{add_thousand_sep(entry['BENIGN'])}}} & "
        latex_code += f"\\multirow{{{len(anomaly_breakdown)}}}{{*}}{{{add_thousand_sep(entry['ANOMALY'])}}} & "
    else:
        # Only one type of anomaly, no need for multirow
        latex_code += f"{ds_label} & {add_thousand_sep(entry['TOTAL'])} & "
        latex_code += f"{add_thousand_sep(entry['BENIGN'])} & {add_thousand_sep(entry['ANOMALY'])} & "

    # Add rows for the anomaly breakdown
    for i, anomaly in enumerate(anomaly_breakdown):
        if i > 0:  # Not the first entry, so we need to start a new table row
            latex_code += " & & & & "
        latex_code += f"{anomaly['Anomaly Type']} & {add_thousand_sep(anomaly['Count'])} & "
        latex_code += f"{add_thousand_sep(anomaly['Min Packets'])} & {add_thousand_sep(anomaly['Mean Packets'])} & "
        latex_code += f"{add_thousand_sep(anomaly['Max Packets'])} \\\\\n"

    # Add a midrule after each DS block if there are multiple anomalies
    latex_code += "\\midrule\n\n"

# Close the LaTeX table structure
latex_code += "\\bottomrule\n\\end{tabular}\n\\end{table*}"

# Display the generated LaTeX code (for testing purposes)
print(latex_code)

with open('results/fd_anomaly_distribution_table.tex', 'w') as file:
    file.write(latex_code)


\begin{table*}[htbp]
\scriptsize
\centering
\caption{Your Table Caption}
\renewcommand{\arraystretch}{0.6} % Reduce spacing
\begin{tabular}{lrrrrrrrr}
\toprule
\textbf{DS} & \textbf{TOTAL} & \textbf{BENIGN} & \textbf{ANOMALY} & \textbf{Anomaly Type} & \textbf{Count} & \textbf{Min Pckts.} & \textbf{Mean Pckts.} & \textbf{Max Pckts.} \\
\midrule
\multirow{4}{*}{CF} & \multirow{4}{*}{502 350} & \multirow{4}{*}{326 363} & \multirow{4}{*}{175 987} & DoS GoldenEye & 7 917 & 2 & 12.14 & 30 \\
 & & & & DoS Hulk & 158 680 & 2 & 9.20 & 27 \\
 & & & & DoS Slowhttptest & 3 707 & 1 & 4.72 & 35 \\
 & & & & DoS Slowloris & 5 683 & 2 & 7.36 & 27 \\
\midrule

\multirow{3}{*}{FD=5ms} & \multirow{3}{*}{7 693} & \multirow{3}{*}{729} & \multirow{3}{*}{6 964} & DoS GoldenEye & 270 & 3 & 7.11 & 15 \\
 & & & & DoS Hulk & 5 917 & 3 & 4.03 & 10 \\
 & & & & DoS Slowhttptest & 777 & 2 & 3.98 & 4 \\
\midrule

\multirow{2}{*}{FD=10ms} & \multirow{2}{*}{82 521} & \multirow{2}{*}{1 093} & \multirow{2}{*}{81 428} & D